# Spark scaling and performance

- https://www.youtube.com/watch?v=JoQ8m-kM_ZY 


## Scaling Spark Driver

Spark Driver is an entity that contains the `SparkContext`.

Some important configuration tunnig of the spark driver are

- Allowing **dynamic executor allocation**. This allows an Spark job to add and remove executors on the fly. 

```
spark.dynamicAllocation.enabled = True
spark.dynamicAllocation.executorIdleTimeout = 2m
spark.dynamicAllocation.minExecutors = 1
spark.dynamicAllocation.maxExecutors = 1000
```

## Scaling Spark Executor


Executor memory is divided into 4 sections:

- Shuffle memory: Used to shuffle the buffer internal memory.
    - As you run out of shuffle memory the data is stored into disk.
- User memory: Used for user specified data structures.
    - 
- Reserved Memory:
- Memory Buffer

#### Tuning `spark.memory.fraction` for Shuffle Memory and User Memory
Notice that Shuffle memory and User Memory are configured by a single definable parameter **`spark.memory.fraction`**. By default 40 % of the executor memory is given to the User Memory.

```
ShuffleMemory = spark.memory.fraction*(spark.executor.memory-300MB)
UserMemory = (1-spark.memory.fraction)*(spark.executor.memory-300MB)
```

#### Tuning `OffHeap` memory

```
spark.memory.offHeap.enable = true
spark.memory.offHeap.size = 3g
spark.executor.memory     = 3g
```


#### Tuning `GC` memory

Garbage collection tuning is important. 

Parallel GC can be enabled using
```
spark.executor.extraJavaOptions = -XX:OarakkekGTCThreads=4 -XX:+UseParallelGC
```

#### Tuning for disk I/O


```
spark.shuffle.file.buffer = 1MB
spark.unsafe.sorter.spill.reader.buffer.size = 1MB
```


#### Tuning for disk I/O, block size compression

```
spark.io.compression.lz4.blockSize = 512KB
```

## Scaling External Shuffle Service

Cache Index Files on Shuffle Server

- Tune shuffle service worker thread and backlog
```
spark.shuffle.io.serverThreads = 128
spark.shuffle.io.backLog = 8192
```

- Configurable shuffle registration timeout and retry
```
spark.shuffle.registration.timeout = 2m
spark.shuffle.registraion.maxAttempts = 5
```

# Application tunning 

Automatically tune hyperparameters of jobs.

## Auto tunning of mapper and reducer

- Heuristics based approach based on table input size:

    - Max cap due to the constrian of the scalability of shuffle service and drivers
    - Min cap due to the minimum guarantee of resources to user's job
    
## Tools to debug spark jobs


- Spark UI metrics: Useful to break down different stages.
- Flame Graph: Stack trace on the worker nodes
    - https://github.com/spektom/spark-flamegraph
    
- Scuba: (Facebook internal tool)
    - https://research.fb.com/wp-content/uploads/2016/11/scuba-diving-into-data-at-facebook.pdf